In [ ]:
from system import System
s = System()

In [ ]:
from aalpy.base import SUL
from aalpy.learning_algs import run_stochastic_Lstar
from aalpy.oracles import  RandomWordEqOracle
from aalpy.utils import visualize_automaton

BOUND = 100 
SELECTED = [
    "command_executed",
    "current_revenue",
    "client_quantity"
]

class SystemSUL(SUL):
    def __init__(self):
        super().__init__()
        self.system = System()
        
        # self.alphabet = []
        # for action in self.system.alphabet():
        #     self.alphabet.append(action)

        self.alphabet = ["run"]

    def pre(self):
        self.system.reset()

    def post(self):
        pass

    def step(self, action):
        if action is not None:
            return self.system.enact_strategy(selected_components=SELECTED ,bound=BOUND)
            #return self.system.action(action, repr=True)

In [ ]:
MAX_ROUNDS = 50

sul = SystemSUL() 
alphabet = sul.alphabet
eq_oracle = RandomWordEqOracle(alphabet, sul, num_walks=10000, min_walk_len=10, max_walk_len=1000, reset_after_cex=True)
learned_mdp = run_stochastic_Lstar(alphabet, sul, eq_oracle, n_c=30, n_resample=200, min_rounds=1, max_rounds=MAX_ROUNDS, print_level=2)

#decomment to visualize the automaton
#visualize_automaton(learned_mdp, display_same_state_trans=True)

In [ ]:
learned_mdp.save('learned_mdp')

In [ ]:
from parse_dot import parse_dot_file, create_transition_matrix, save_matrix_to_csv, save_node_labels
import numpy as np

# Define input and output files
input_file = "learned_mdp.dot"
matrix_output = "transition_matrix.csv"
labels_output = "node_labels.txt"

# Parse the DOT file
print(f"Parsing {input_file}...")
nodes, transitions = parse_dot_file(input_file)
print(f"Found {len(nodes)} nodes and {len(transitions)} transitions.")

# Create the transition matrix
print("Creating transition matrix...")
matrix, node_ids = create_transition_matrix(nodes, transitions)

# Save to CSV
save_matrix_to_csv(matrix, matrix_output)

# Save node labels
save_node_labels(nodes, labels_output)

# Generate summary information
print("\nMatrix Summary:")
print(f"Shape: {matrix.shape}")

# Count non-zero transitions
non_zero = np.count_nonzero(matrix)
print(f"Non-zero transitions: {non_zero}")

# Calculate sparsity
sparsity = 1.0 - (non_zero / (matrix.shape[0] * matrix.shape[1]))
print(f"Sparsity: {sparsity:.4f} ({sparsity*100:.2f}%)")

# Find nodes with most outgoing transitions
outgoing_counts = np.count_nonzero(matrix, axis=1)
max_outgoing = np.max(outgoing_counts)
nodes_with_max_outgoing = np.where(outgoing_counts == max_outgoing)[0]
print(f"Max outgoing transitions: {max_outgoing} (from nodes {', '.join(map(str, nodes_with_max_outgoing))})")

In [ ]:
from completion_states import generate_completion_state_labels

dot_file = "learned_mdp.dot"
    
labels = generate_completion_state_labels(dot_file)
    
print("state_labels = {")
for state_id, label in labels.items():
    print(f"    {state_id}: \"{label}\",")
print("}")

In [ ]:
import pandas as pd

state_labels = {int(k):v for k,v in   generate_completion_state_labels(dot_file).items()}

# Example configuration - update with your specific parameters
input_file = "transition_matrix.csv"
source_node = 0
target_states = list(state_labels.keys())  # Example target states
max_steps = 50

# Optional: provide human-readable labels for the states

# Load the transition matrix
matrix = pd.read_csv(input_file, index_col=0).values

In [ ]:
from analysis import compute_step_by_step_probabilities, compute_cumulative_probabilities, analyze_predecessors, plot_with_predecessors

time_series = compute_step_by_step_probabilities(matrix, source_node, target_states, max_steps)
cumulative_series = compute_cumulative_probabilities(matrix, source_node, target_states, max_steps)

# Analyze predecessors
print("Analyzing predecessors...")
predecessors = analyze_predecessors(matrix, target_states, state_labels)

# Create combined plot with predecessors
print("Creating visualization...")
plot_with_predecessors(time_series, cumulative_series, matrix, source_node, 
                      target_states, predecessors, max_steps, state_labels)

In [ ]:
from graphviz import Source
from IPython.display import display
from parse_dot import filter_dot_file

# Define target states

# Filter the DOT file and get the content
filtered_dot_content = filter_dot_file("learned_mdp.dot", target_states)

# Render the graph directly
graph = Source(filtered_dot_content)
display(graph)

# Optionally save the filtered DOT file
with open("filtered_mdp.dot", "w") as f:
    f.write(filtered_dot_content)